In [58]:
import pandas as pd
import numpy as np

In [59]:
data = pd.read_csv("./datasets/not_null.csv")

In [60]:
data.head()

,class,tweet
0,1,!!! As a woman you shouldn't complain about cl...
1,0,!!!!! boy dats cold...tyga dwn bad for cuffin ...
2,0,!!!!!!! Dawg!!!! You ever fuck a bitch and she...
3,0,!!!!!!!!! she look like a tranny
4,0,!!!!!!!!!!!!! The shit you hear about me might...


In [61]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['tweet'], data['class'], test_size=0.3, random_state=1)
x_train.shape

(31270,)

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x_train)
X_train_counts.shape

(31270, 29828)

In [63]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(31270, 29828)

In [64]:
# Machine Learning
# Training Naive Bayes (NB) classifier on training data.
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [65]:
# Building a pipeline: We can write less code and do all of the above, by building a pipeline as follows:
# The names ‘vect’ , ‘tfidf’ and ‘clf’ are arbitrary but will be used later.
# We will be using the 'text_clf' going forward.
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(x_train, y_train)

In [66]:
import numpy as np
predicted = text_clf.predict(x_test)
np.mean(predicted == y_test)

0.9305327563050291

In [67]:
# Training Support Vector Machines - SVM and calculating its performance

from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=10, random_state=42))])

text_clf_svm = text_clf_svm.fit(x_train, y_train)
predicted_svm = text_clf_svm.predict(x_test)
np.mean(predicted_svm == y_test)

0.9223996418445009

In [68]:
# Grid Search
# Here, we are creating a list of parameters for which we would like to do performance tuning. 
# All the parameters name start with the classifier name (remember the arbitrary name we gave). 
# E.g. vect__ngram_range; here we are telling to use unigram and bigrams and choose the one which is optimal.

from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

In [69]:
# Next, we create an instance of the grid search by passing the classifier, parameters 
# and n_jobs=-1 which tells to use multiple cores from user machine.

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1, cv=5)
gs_clf = gs_clf.fit(x_train, y_train)

/home/sergi/venv/mamut/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [70]:
# To see the best mean score and the params, run the following code

gs_clf.best_params_
gs_clf.best_score_

# Output for above should be: The accuracy has now increased to ~90.6% for the NB classifier (not so naive anymore! 😄)
# and the corresponding parameters are {‘clf__alpha’: 0.01, ‘tfidf__use_idf’: True, ‘vect__ngram_range’: (1, 2)}.

0.9278221937959705

In [71]:
predicted_gs_clf = gs_clf.predict(x_test)

np.mean(predicted_gs_clf == y_test)

0.9303089091180421

In [ ]:
# Similarly doing grid search for SVM
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}

gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1, cv=5)
gs_clf_svm = gs_clf_svm.fit(x_train, y_train)


gs_clf_svm.best_score_
gs_clf_svm.best_params_

/home/sergi/venv/mamut/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [ ]:
gs_clf_svm.best_score_

In [ ]:
predicted_gs_clf_svm = gs_clf_svm.predict(x_test)

np.mean(predicted_gs_clf_svm == y_test)

In [ ]:
# NLTK
# Removing stop words
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])

In [ ]:
# Stemming Code

import nltk

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect), ('tfidf', TfidfTransformer()), 
                             ('mnb', MultinomialNB(fit_prior=False))])

text_mnb_stemmed = text_mnb_stemmed.fit(x_train, y_train)

predicted_mnb_stemmed = text_mnb_stemmed.predict(x_test)

np.mean(predicted_mnb_stemmed == y_test)

In [ ]:
text_svm_mnb_stemmed = Pipeline([('vect', stemmed_count_vect), ('tfidf', TfidfTransformer()), 
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=10, random_state=42))])

text_svm_mnb_stemmed = text_svm_mnb_stemmed.fit(x_train, y_train)

predicted_svm_mnb_stemmed = text_svm_mnb_stemmed.predict(x_test)

np.mean(predicted_svm_mnb_stemmed == y_test)

In [ ]:
data.head()

In [ ]:
len(data)

In [ ]:
20620 - 4163

In [ ]:
text_svm_mnb_stemmed.predict(["Woman should stay in the kitchen"])

In [ ]:
import joblib as jb
jb.dump(text_svm_mnb_stemmed, 'clf.joblib')